# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.core import Pipeline

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
project_name = 'azureml-spaceship-titanic'
project_folder = os.getcwd().split(project_name)[0] + project_name

In [11]:
import sys
sys.path.append(project_folder)

from src.pipelines.preprocess import preprocess_data

df = pd.read_csv('../data/01_raw/train.csv')
processed_df = preprocess_data(df).drop(columns=["PassengerId"])


In [12]:
dataset_name = "Spaceship_Dataset"
description_text = "Data to predict which passengers are transported to an alternate dimension"

# Create AML Dataset and register it into Workspace
dataset = Dataset.Tabular.register_pandas_dataframe(processed_df, ws.get_default_datastore(), dataset_name)   
#Register Dataset in Workspace
dataset = dataset.register(workspace=ws,
                                name=dataset_name,
                                description=description_text,
                                create_new_version=True
                        )


df = dataset.to_pandas_dataframe()
df.describe()

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/a4b6936c-442b-4bac-864c-198d247c09b0/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


RoomService     FoodCourt  ShoppingMall           Spa        VRDeck  \
count   8512.000000   8510.000000   8485.000000   8510.000000   8505.000000   
mean     224.687617    458.077203    173.729169    311.138778    304.854791   
std      666.717663   1611.489240    604.696458   1136.705535   1145.717189   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%       47.000000     76.000000     27.000000     59.000000     46.000000   
max    14327.000000  29813.000000  23492.000000  22408.000000  24133.000000   

       People_in_Cabin_Num  People_in_Cabin_Deck  Family_Size   Group_Size  
count          8494.000000           8494.000000  8493.000000  8693.000000  
mean              8.037203           1952.105957     5.428117     2.035546  
std               5.214669            963.741893     2.891901     1.596347  
min               1.000000              5.000000     1.000000     1.000000  
25%               4.000000            779.000000     3.000000     1.000000  
50%               7.000000           2559.000000     5.000000     1.000000  
75%              12.000000           2794.000000     7.000000     3.000000  
max              28.000000           2794.000000    18.000000     8.000000

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Transported",   
                             path = project_folder + '/automl-pipeline',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [14]:
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.widgets import RunDetails

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)


pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

pipeline_run = experiment.submit(pipeline)
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()


Created step automl_module [e3f1265b][ac048c96-3ccc-4b76-bb16-0680f8ec9077], (This step will run and generate new outputs)
Submitted PipelineRun 95e42c09-466d-4a3f-9818-118135873222
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/95e42c09-466d-4a3f-9818-118135873222?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-250931/workspaces/quick-starts-ws-250931&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 95e42c09-466d-4a3f-9818-118135873222
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/95e42c09-466d-4a3f-9818-118135873222?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-250931/workspaces/quick-starts-ws-250931&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 6bb99c33-2d80-47ab-aee0-157c61c494b2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6bb99c33-2d80-47ab-aee0-157c61c494b2?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-250931/workspaces/quick-starts-ws-250931&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabl

'Finished'

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
import json

metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

Downloaded azureml/6bb99c33-2d80-47ab-aee0-157c61c494b2/metrics_data, 1 files out of an estimated total of 1


6bb99c33-2d80-47ab-aee0-157c61c494b2_9  \
weighted_accuracy                                  [0.7368115160608307]   
AUC_micro                                          [0.8086904158169231]   
precision_score_micro                              [0.7376047772860067]   
balanced_accuracy                                  [0.7383988256755011]   
log_loss                                           [0.5751922936880788]   
recall_score_macro                                 [0.7383988256755011]   
AUC_weighted                                       [0.8077551141351792]   
precision_score_macro                              [0.7482531927109872]   
recall_score_weighted                              [0.7376047772860067]   
average_precision_score_micro                      [0.8073004870647242]   
accuracy                                           [0.7376047772860067]   
f1_score_micro                                     [0.7376047772860067]   
f1_score_macro                                     [0.7350817545895257]   
average_precision_score_weighted                   [0.7982350638715339]   
average_precision_score_macro                      [0.7979800454580991]   
recall_score_micro                                 [0.7376047772860067]   
AUC_macro                                           [0.807755114135179]   
matthews_correlation                              [0.48654770716028467]   
f1_score_weighted                                  [0.7349386945842366]   
precision_score_weighted                           [0.7487840114888646]   
norm_macro_recall                                   [0.476797651351002]   

                                 6bb99c33-2d80-47ab-aee0-157c61c494b2_2  \
weighted_accuracy                                   [0.737882179505414]   
AUC_micro                                          [0.8292465103643746]   
precision_score_micro                              [0.7388706926459626]   
balanced_accuracy                                  [0.7398601167171766]   
log_loss                                           [0.5070073230419038]   
recall_score_macro                                 [0.7398601167171766]   
AUC_weighted                                       [0.8330764219736091]   
precision_score_macro                               [0.750554023107146]   
recall_score_weighted                              [0.7388706926459626]   
average_precision_score_micro                      [0.8319711373223694]   
accuracy                                           [0.7388706926459626]   
f1_score_micro                                     [0.7388706926459626]   
f1_score_macro                                     [0.7362336512481521]   
average_precision_score_weighted                   [0.8281984050435603]   
average_precision_score_macro                      [0.8278866168618809]   
recall_score_micro                                 [0.7388706926459626]   
AUC_macro                                          [0.8330764219736091]   
matthews_correlation                              [0.49029305404776236]   
f1_score_weighted                                  [0.7360339163987962]   
precision_score_weighted                           [0.7511817450920452]   
norm_macro_recall                                  [0.4797202334343531]   

                                 6bb99c33-2d80-47ab-aee0-157c61c494b2_12  \
weighted_accuracy                                   [0.8030715042964353]   
AUC_micro                                           [0.8928054315265345]   
precision_score_micro                               [0.8030599942397755]   
balanced_accuracy                                    [0.803048501893762]   
log_loss                                           [0.42135740405926175]   
recall_score_macro                                   [0.803048501893762]   
AUC_weighted                                        [0.8900658520042888]   
precision_score_macro                                 [0.80349918601929]   
recall_score_weighted                          

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
import pickle

# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

Downloaded azureml/6bb99c33-2d80-47ab-aee0-157c61c494b2/model_data, 1 files out of an estimated total of 1


PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('35', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.3966666666666666, learning_rate=0.06842421052631578, max_bin=360, max_depth=2, min_child_weight=8, min_data_in_leaf=1e-05, min_split_gain=0.9473684210526315, n_estimators=600, n_jobs=1, num_leaves=200, problem_info=ProblemInfo(gpu_training

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Deployed from Studio



TODO: In the cell below, send a request to the web service you deployed to test it.

In [29]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://091674f3-7d52-4713-a563-29fdf9673dcd.westeurope.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = '2cloAPK95e4LlPLSINN0HXiMNWGOcJb5'

# Two sets of data to score, so we get two results back
data = {
    "Inputs": {
        "data":
            [
              {
        "HomePlanet": "Europa",
        "CryoSleep": "False",
        "Destination": "TRAPPIST-1e",
        "VIP": "False",
        "RoomService": 109.00,
        "FoodCourt": 1000,
        "ShoppingMall": 25.0,
        "Spa": 200.0,
        "VRDeck": 2.0,
        "Cabin_Deck": "B",
        "Cabin_Side": "P",
        "Cabin_Region": "A",
        "People_in_Cabin_Num": 14,
        "People_in_Cabin_Deck": 700,
        "Family_Size": 4,
        "Group_Size": 2,
        "Age_Cat": "Pre_Adult"
        }
      ]
    },
    "GlobalParameters": {
        "method": "predict"
    }
}
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


{'Results': [True]}


TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
%%writefile logs.py
from azureml.core import Workspace, Webservice
from azureml.exceptions import WebserviceException
import time

# Load the Azure ML workspace
ws = Workspace.from_config()

# Name of the web service
service_name = 'web-service-automl'

# Function to attempt to delete the service
def try_delete_service(service):
    try:
        service.delete()
        print("Web service deleted successfully.")
        return True
    except WebserviceException as e:
        print(f"Error deleting web service: {str(e)}")
        return False

# Get a reference to the web service
service = Webservice(workspace=ws, name=service_name)

# Print logs
print("Web service logs:")
print(service.get_logs())

# Wait for deployment operation to complete
try:
    service.wait_for_deployment()
except WebserviceException as e:
    # Ignore the exception related to the operation status
    if "No operation endpoint" not in str(e) and "Long running operation information not known" not in str(e):
        raise

# Attempt to delete the deployed web service with retries
max_retries = 3
retry_count = 0

while retry_count < max_retries:
    if try_delete_service(service):
        break
    else:
        retry_count += 1
        print(f"Retrying deletion (attempt {retry_count}/{max_retries})...")
        time.sleep(10)  # Wait for 10 seconds before retrying

if retry_count == max_retries:
    print("Max retries reached. Unable to delete the web service.")


Overwriting logs.py


In [31]:
!python logs.py

Web service logs:
2024-01-29T02:19:59,463994100+00:00 - rsyslog/run 
2024-01-29T02:19:59,473020200+00:00 - gunicorn/run 
2024-01-29T02:19:59,481847500+00:00 | gunicorn/run | 
2024-01-29T02:19:59,486636100+00:00 | gunicorn/run | ###############################################
2024-01-29T02:19:59,493631400+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-01-29T02:19:59,499912100+00:00 | gunicorn/run | ###############################################
2024-01-29T02:19:59,504231000+00:00 | gunicorn/run | 
2024-01-29T02:19:59,520512700+00:00 | gunicorn/run | 
2024-01-29T02:19:59,538671200+00:00 - nginx/run 
2024-01-29T02:19:59,545640900+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20231216.v3
2024-01-29T02:19:59,547958300+00:00 | gunicorn/run | 
2024-01-29T02:19:59,550264400+00:00 | gunicorn/run | 
2024-01-29T02:19:59,556724100+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
